# 2.2 Negation as Failure

Negation as Failure (NAF) searches for an absence of data - inferring new facts where a pattern is missing.

There are two critical components to understand when using NAF with Datalog rules, **NOT** and **NOT EXISTS**.

## NOT

When used alone, **NOT** checks for a pattern to be false.

### The Closed World Assumption

RDFox follows the closed world assumption, which assumes statements to be false if they cannot be proven true.

This means that the pattern can either be provably false or not present in the data at all to trigger the inference.

## NOT EXISTS

Subtly different from **NOT**, **NOT EXISTS** allows us to infer new facts based on missing data.

**NOT EXISTS** searches the data store for the described pattern. If the returned result set is empty, **NOT EXISTS** returns True and vice versa.

# Exercise 1

Rule the following example code cell to see the different implications of **NOT** vs **NOT EXISTS**.

In [23]:
naf_data = """
@prefix : <https://rdfox.com/example#> .

:justTrueNode a :Thing ;
    :hasValue true .

:justFalseNode a :Thing ;
    :hasValue false .

:bothValuesNode a :Thing ;
    :hasValue true ;
    :hasValue false .

:noValueNode a :Thing .

"""

In [25]:
naf_rules = """
prefix : <https://rdfox.com/example#>

[?x, a, :FalseThing] :-
    [?x, a, :Thing] ,
    NOT [?x, :hasValue, true] .

[?x, a, :NoValueThing] :-
    [?x, a, :Thing] ,
    NOT EXISTS ?v IN (
        [?x, :hasValue, ?v]
    ) .

"""

In [37]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=naf_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=naf_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue explicit values select query
ev_sparql_text = "SELECT * WHERE { ?S a :Thing . OPTIONAL{ SELECT ?S ?v WHERE { ?S a :Thing; :hasValue ?v } } }"
ev_response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": ev_sparql_text})
assert_response_ok(ev_response, "Failed to run ft select query.")
print('=== Explicit Values of Things ===')
print(ev_response.text)

# Issue FalseThing select query
ft_sparql_text = "SELECT * WHERE { ?S a :FalseThing . OPTIONAL{ SELECT ?S ?v WHERE { ?S a :FalseThing; :hasValue ?v } } }"
ft_response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": ft_sparql_text})
assert_response_ok(ft_response, "Failed to run ft select query.")
print('\n=== Inferred Things from NOT [?x, :hasValue, true] ===')
print(ft_response.text)

# Issue NoValueThing select query
nvt_sparql_text = "SELECT ?S ?v WHERE { ?S a :NoValueThing . OPTIONAL{ SELECT ?S ?v WHERE { ?S a :NoValueThing; :hasValue ?v } } }"
nvt_response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": nvt_sparql_text})
assert_response_ok(nvt_response, "Failed to run nvt select query.")
print('\n=== Inferred Things from NOT EXISTS ?v IN ([?x, :hasValue, ?v]) . ===')
print(nvt_response.text)


=== Explicit Values of Things ===
?S	?v
<https://rdfox.com/example#justTrueNode>	true
<https://rdfox.com/example#noValueNode>	
<https://rdfox.com/example#bothValuesNode>	true
<https://rdfox.com/example#bothValuesNode>	false
<https://rdfox.com/example#justFalseNode>	false


=== Inferred Things from NOT [?x, :hasValue, true] ===
?S	?v
<https://rdfox.com/example#noValueNode>	
<https://rdfox.com/example#justFalseNode>	false


=== Inferred Things from NOT EXISTS ?v IN ([?x, :hasValue, ?v]) . ===
?S	?v
<https://rdfox.com/example#noValueNode>	



# Exercise 2

Open **NAF.dlog** in the **rules folder** and fill in the blanks to achieve... (EXERCISE 2 NOT DONE YET)

Rule the code cell below to import the rules **NAF.dlog**.

In [3]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Read rules from external file
with open("../rules/NAF.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)


# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Catch errors
assert_response_ok(response, "Failed to add rule.")

print(response)

Results:  <Response [200]>


### Check your work

Now run the query below to verify the results.

Is it what you expected?

In [4]:
# Read query from external file
with open("../queries/star.rq", "r") as query_file:
    query = query_file.read()

# Get response
queryResponse = requests.get(rdfox_server + "/datastores/default/sparql", params={"query": query})

# Catch errors
assert_response_ok(queryResponse, "Failed to run select query.")

print("Results:\n", queryResponse.text)

Results: 
 ?s	?p	?o
<https://rdfox.com/example#a>	<https://rdfox.com/example#b>	<https://rdfox.com/example#c>



## Incremental Retraction

NAF is an incredibly powerful tool and raises an potentially thorny question when combined with the Incremental Reasoning of RDFox.

What happens when new data is added that fills the gap of an absent triple, whose absence was leading to the inference of another fact?

Well, the previously inferred fact will have to be retracted.

Try importing some new data that contains something from the **NOT EXISTS** atom and see what happens.

In [19]:
# Issue insert
sparql_insert = "prefix : <https://rdfox.com/example#> INSERT { ?s :hasProp 'Property' } WHERE { ?s :b ?o }"
response = requests.post(
    rdfox_server + "/datastores/default/sparql", data={"update": sparql_insert})
assert_response_ok(response, "Failed to insert fact via sparql.")

# Get response
queryResponse = requests.get(rdfox_server + "/datastores/default/sparql", params={"query": query})

# Catch errors
assert_response_ok(queryResponse, "Failed to run select query.")

print("Results: ", queryResponse.text)

Results:  ?s	?p	?o
<https://rdfox.com/example#a>	<https://rdfox.com/example#b>	<https://rdfox.com/example#c>
<https://rdfox.com/example#c>	<https://rdfox.com/example#wooooow>	<https://rdfox.com/example#a>
<https://rdfox.com/example#a>	<https://rdfox.com/example#wooooow>	<https://rdfox.com/example#c>
<https://rdfox.com/example#a>	<https://rdfox.com/example#excellent>	<https://rdfox.com/example#c>
<https://rdfox.com/example#a>	<https://rdfox.com/example#hasProp>	"Property"
<https://rdfox.com/example#a>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<https://rdfox.com/example#Class>



## Cycles in Rules

A rule, or set of rules, cannot be imported if they create cyclic logic, meaning an inferred triples change the conditions that led to their inference.

This is known as a stratification error - the rule set cannot be stratified (logically chained in a way that terminates).

### Where can cycles appear?

It can be easy to write rules that cannot be stratified when using Aggregation and Negation.

This behavior can be difficult to spot so RDFox will prevent you from importing such rules and will tell you where the cycle occurs.

With a simple example, it's easy to see. Take this rule, can you see the problem?

In [6]:
datalog_rule = """
prefix : <https://rdfox.com/example#>

[?x, a, :Class] :-
    [?x, a, ?c] ,
    NOT [?x, a, :Class] .
"""

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)

# Catch errors
assert_response_ok(response, "Failed to add rule.")

print(response)

Exception: Failed to add rule.
Status received=400
RuleCompilationException: The program is not stratified because these components of the dependency graph contain cycles through negation and/or aggregation:
======== COMPONENT 1 ========
    <https://rdfox.com/example#Class>[?s] :- <https://rdfox.com/example#hasProp>[?s, ?o], NOT <https://rdfox.com/example#hasProp>[?s, "property"] .
    <https://rdfox.com/example#Class>[?x] :- rdf:type[?x, ?c], NOT <https://rdfox.com/example#Class>[?x] .
    <https://rdfox.com/example#DifferentClass>[?s] :- <https://rdfox.com/example#hasProp>[?s, ?o], NOT EXISTS ?o IN <https://rdfox.com/example#hasProp>[?s, ?o] .
========================================================================================================================


## Industry Examples

examples

examples